# Galakser 

In [475]:
%%HTML
<style>
p {font-size:1.2em;
  font-family: Georgia, serif};
</style>

In [476]:
import pandas as pd

In [477]:
bøker = pd.read_csv("../../ngram_2021_coordination/coord-bok.csv", header = None)

In [478]:
bøker.columns = ['lang', 'first', 'coord', 'second', 'freq']

In [479]:
bøker[bøker['first'] == '.'].head(2)

,lang,first,coord,second,freq
8927,nno,.,og,"0,75",4
8928,nno,.,og,09.01.2010,4


In [480]:
"a".isalpha()

True

In [481]:
bøker_words = bøker[bøker['first'].map(lambda x: x.isalpha())][bøker['second'].map(lambda x: x.isalpha())]

/tmp/ipykernel_154307/342431437.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  bøker_words = bøker[bøker['first'].map(lambda x: x.isalpha())][bøker['second'].map(lambda x: x.isalpha())]


In [482]:
bøker_words

,lang,first,coord,second,freq
15380,nno,A,og,Fig,4
15381,nno,A,og,Marianna,4
15382,nno,A,og,VB,4
15383,nno,A,og,alt,4
15384,nno,A,og,anten,4
...,...,...,...,...,...
4752037,nob,mer,og,mer,593743
4752038,nob,rett,og,slett,597504
4752052,nob,av,og,til,984899
4752055,nob,først,og,fremst,1637971


In [483]:
coord = bøker_words.groupby(['first', 'second']).sum()

In [484]:
from math import log, sqrt

In [81]:
#coord['weight'] = coord['freq']*coord['pmi']

In a position "$x\textrm{ and }y$" the part frequencies of $x$, $x_f$, are used in computing $\frac{p(y|x)}{p(y)}$. All proportions are computed from that particular position. So the what is computed
is

1. $p(x) = \frac{x_f}{\sigma(x_f)}$ and
1. $p(y|x) = \frac{y_f^x}{\sigma(x_f)}$. 

Here $y_f^x$ is the the frequency of $y$ in the context $x$. The complete formula to be taken to pandas is then: 

$p(x|y) = \frac{x^y}{\sigma(y)}$ 

$p(y) = \frac{\sigma(y)}{t}$


In [175]:
cond = lambda x, level = 0:(coord.xs(x, level = level)/coord.xs(x, level = level).sum())

In [176]:
summation = lambda x, level = 0: coord.xs(x, level = level).sum()

In [186]:
summation('Hamsun', 0)

freq    4756
dtype: int64

In [193]:
triples = pd.DataFrame([(x, summation(x, 0)) for x in set(coord.index.get_level_values(level = 0))], columns = ['word', 'as_first'])

In [194]:
tuples_second = pd.DataFrame([(x, summation(x, 1)) for x in set(coord.index.get_level_values(level = 1))], columns = ['word', 'as_second'])

In [222]:
tuples_second = tuples_second.set_index('word')

In [224]:
tuples_second = tuples_second.fillna(0).applymap(int)

In [237]:
tuples_second.loc['i']

as_second    2809784
Name: i, dtype: int64

In [227]:
totals_second = tuples_second.sum()
totals_second

as_second    231478939
dtype: int64

In [225]:
#tuples_second.to_csv("second_position_frequencies.csv", header = True)

## Fix values and such

In [200]:
galaxy_freqs = pd.concat([triples.set_index('word'), tuples_second.set_index('word')], axis = 1)

In [208]:
galaxy_freqs = galaxy_freqs.fillna(0).applymap(int)

In [220]:
#galaxy_freqs.to_csv('galaxy_frequencies.csv', header = True)

## Compute pmi

In [219]:
galaxy_freqs.loc['spiser']

as_first     17552
as_second    42501
Name: spiser, dtype: int64

In [230]:
tuples_second.loc['spiser']

as_second    42501
Name: spiser, dtype: int64

In [387]:
def pmi_series(df = None, tuples_second = None, word = None, cutoff = 1):

    # conditional probabilities in series
    series = (df.loc[word]/df.loc[word].sum())['freq']
    second_series = df.loc[word]
    # divided by general probabilites (not normalized!!)
    return second_series['freq']/pd.Series([tuples_second.loc[x].values[0] for x in second_series.index], index = second_series.index)
    

## A

In [430]:
a = pmi_series(df = coord, tuples_second = tuples_second, word = "diktatur", cutoff = 0).sort_values(ascending = False).head(50)

In [ ]:
res = pd.DataFrame()
i = 0
for word in galaxy_freqs.index:
    i += 1
    if i % 100000 == 0:
        print('antall ord', i)
    if galaxy_freqs.loc["word"].as_first > 0:
        s = pd.DataFrame(pmi_series(df = coord, tuples_second = tuples_second, word = word, cutoff = 0), columns = ['pmi'])
        s.index = pd.MultiIndex.from_tuples([(word, i) for i in s.index], names = ['first', 'second'])
        res = pd.concat([res, pd.DataFrame(s)])

In [ ]:
res.merge(coord, left_index = True, right_index = True)

In [ ]:
res.to_csv('galaxy_books.csv')

In [435]:
c['relative'] = (c/c.sum())['freq']

In [438]:
#pd.Series([tuples_second.loc[x].values[0] for x in c.index], index = c.index)

In [433]:
c['score'] = c.freq*c.pmi

In [436]:
c.sort_values(by = "score", ascending = False).head(50)

,freq,pmi,score,relative
second,,,,
masseaktionen,13,1.000000,13.000000,0.011404
masseaksjonen,11,0.458333,5.041667,0.009649
sovjetmakten,7,0.500000,3.500000,0.006140
demokrati,260,0.010727,2.788894,0.228070
tyranni,24,0.048780,1.170732,0.021053
undertrykkelse,79,0.009406,0.743065,0.069298
terror,61,0.011540,0.703935,0.053509
ensretting,13,0.043919,0.570946,0.011404
klassekampen,7,0.057377,0.401639,0.006140


In [400]:
coord.loc['sitter'].sort_values(by = 'freq')

,freq
second,
googler,5
nyder,5
vender,5
knar,5
bæsj,5
...,...
leser,5096
tenker,5681
skriver,6023


In [276]:
series = (coord.loc['spiser']/coord.loc['spiser'].sum())['freq']

In [279]:
second_series = coord.loc['spiser']
second_series['freq']

second
Eleseus        8
andre         10
arbeider      54
at            52
av            10
            ... 
vi            47
vil           11
vokser       159
å              6
ødelegger     20
Name: freq, Length: 137, dtype: int64

In [303]:
coord.loc['spiser'].sort_values(by = 'freq', ascending = False).head(30)

,freq
second,
drikker,9341
sover,1681
spiser,691
hvordan,364
hvor,312
lever,297
snakker,248
koser,221
blir,217


In [300]:
pd.Series([tuples_second.loc[x].values[0] for x in second_series.index], index = second_series.index).sort_values(ascending = False).head(30)

second
med        3601726
så         3034962
i          2809784
til        2589892
det        2174440
de         2104022
den        2086177
andre      1713435
ikke       1463778
jeg        1456740
for        1119999
at         1067333
som        1064605
er         1014549
på         1000415
har         955633
se          730018
om          586914
ser         521481
kan         484861
han         445739
blir        360262
går         339951
å           312099
etter       304255
hvordan     301701
hva         299967
dermed      293198
av          285535
hvor        270729
dtype: int64

In [299]:
(second_series['freq']/pd.Series([tuples_second.loc[x].values[0] for x in second_series.index], index = second_series.index)).sort_values(ascending = False).head(30)

second
fordøyer              0.294798
drikker               0.206089
puler                 0.161290
hjertekarsykdommer    0.137255
hygger                0.110420
driter                0.108225
overnatter            0.089431
kvitrer               0.085470
Eleseus               0.072072
bæsjer                0.066667
koser                 0.060565
sover                 0.058324
gotter                0.038462
smatter               0.023739
spiser                0.016258
småprater             0.014925
kler                  0.012417
nyter                 0.011162
vokser                0.010851
hviler                0.010793
prater                0.010160
lever                 0.008963
tygger                0.008696
trener                0.008538
spyr                  0.007194
oppholder             0.006933
slapper               0.005076
nynner                0.004892
kroppens              0.004449
formerer              0.004269
dtype: float64

In [277]:
series

second
Eleseus      0.000456
andre        0.000570
arbeider     0.003077
at           0.002963
av           0.000570
               ...   
vi           0.002678
vil          0.000627
vokser       0.009059
å            0.000342
ødelegger    0.001139
Name: freq, Length: 137, dtype: float64

In [267]:
coord.loc['A']['pmi']  = pmi_series('A')

In [269]:
coord.loc['A']

,freq
second,
A,2052
AA,25
AB,598
AC,8
AN,5
...,...
én,99
ø,14
øienafd,5


In [270]:
pmi_series('abort').sort_values(ascending = False).head(30)

second
prevensjon           9400.898033
for                  5409.639570
eutanasi             3661.402392
homofili             2869.747821
sterilisering        2770.790999
fødsel               2605.862964
en                   2473.920535
skilsmisse           2374.963714
dødfødsel            1913.165214
dødshjelp            1781.222785
seksuelt             1715.251571
fosterdiagnostikk    1682.265964
andre                1682.265964
provosert            1649.280357
homoseksualitet      1550.323535
som                  1484.352321
det                  1451.366714
fosterteknologi      1385.395500
om                   1352.409893
mot                  1352.409893
prostitusjon         1187.481857
at                   1121.510643
den                  1121.510643
barnedrap            1055.539428
aktiv                 989.568214
intrauterin           956.582607
ikke                  824.640178
å                     824.640178
i                     791.654571
de                    791.654571
Nam

In [271]:
coord.loc['abort']['freq'].sort_values(ascending = False).head(30)

second
prevensjon           285
for                  164
eutanasi             111
homofili              87
sterilisering         84
fødsel                79
en                    75
skilsmisse            72
dødfødsel             58
dødshjelp             54
seksuelt              52
fosterdiagnostikk     51
andre                 51
provosert             50
homoseksualitet       47
som                   45
det                   44
fosterteknologi       42
om                    41
mot                   41
prostitusjon          36
at                    34
den                   34
barnedrap             32
aktiv                 30
intrauterin           29
ikke                  25
å                     25
i                     24
de                    24
Name: freq, dtype: int64

In [173]:
cond2('drikker').sort_values(by = 'freq', ascending = False).head(20)

,freq
second,
spiser,0.081893
drikker,0.081610
røyker,0.056673
er,0.041513
spiller,0.038254
snakker,0.028903
sover,0.028762
prater,0.028053
danser,0.027345


In [174]:
coord.xs('drikker', level = 0).sort_values(by = "freq", ascending = False).head(20)

,freq
second,
spiser,578
drikker,576
røyker,400
er,293
spiller,270
snakker,204
sover,203
prater,198
danser,193


In [147]:
coord.loc["smør"].sort_values(by = 'freq', ascending = False).head(50)

,freq
second,
ost,15519
sukker,10167
mel,4132
brød,2679
olje,2647
margarin,2272
egg,1903
en,1750
melk,1497


In [ ]:
def make_graph(word):
    seconds = coord.loc["